In [1]:
import threading

In [3]:
threading.active_count()

8

In [4]:
threading.current_thread()

<_MainThread(MainThread, started 140704447928064)>

In [5]:
threading.enumerate()

[<_MainThread(MainThread, started 140704447928064)>,
 <Thread(IOPub, started daemon 123145485369344)>,
 <Heartbeat(Heartbeat, started daemon 123145502158848)>,
 <Thread(Thread-3, started daemon 123145520021504)>,
 <Thread(Thread-4, started daemon 123145536811008)>,
 <ControlThread(Control, started daemon 123145553600512)>,
 <HistorySavingThread(IPythonHistorySavingThread, started 123145570390016)>,
 <ParentPollerUnix(Thread-2, started daemon 123145587716096)>]

In [6]:
threading.get_ident()

140704447928064

In [7]:
threading.get_native_id()

9083207

In [8]:
main = threading.main_thread()

In [9]:
main

<_MainThread(MainThread, started 140704447928064)>

In [31]:
import time

class DataProcessor(threading.Thread):
    def __init__(self, n):
        self.n = n
        super().__init__()
    
    def run(self):
        print(f"{self.name=}, {self.native_id=}, {self.daemon=}")
        for i in range(self.n):
            time.sleep(1)
            print("after sleep", i)

In [32]:
proc = DataProcessor(7)
proc.start()
time.sleep(3)
print("After start in 2 seconds")
proc.join()
print("After join")

self.name='Thread-11', self.native_id=9089989, self.daemon=False
after sleep 0
after sleep 1
After start in 2 seconds
after sleep 2
after sleep 3
after sleep 4
after sleep 5
after sleep 6
After join


In [17]:
time.sleep(10)

In [45]:
%%time

N = 10 ** 8

def counter(a, b):
    while a < b:
        a += 1

counter(0, N)

CPU times: user 7.6 s, sys: 217 ms, total: 7.81 s
Wall time: 8.37 s


In [44]:
%%time

N_THREADS = 2
COUNT_SIZE = N // N_THREADS

def counter(l, r):
    print(f"[f{threading.current_thread().name=}] {l=}, {r=}")
    while l < r:
        l += 1

threads = [
    threading.Thread(target=counter, args=(i * COUNT_SIZE, (i+1)*COUNT_SIZE), name=f"SuperThread #{i}")
    for i in range(N_THREADS)
]

for th in threads:
    th.start()

for th in threads:
    th.join()

[fthreading.current_thread().name='SuperThread #0'] l=0, r=50000000
[fthreading.current_thread().name='SuperThread #1'] l=50000000, r=100000000
CPU times: user 7.48 s, sys: 259 ms, total: 7.73 s
Wall time: 8.01 s


In [50]:
%%time

N = 100
URL = 'https://ru.wikipedia.org/wiki/Python'
URLS = [URL] * N

import requests

def fetch_url(url):
    resp = requests.get(url)
    return resp
              
def fetch_batch_urls(urls):
    for url in urls:
        fetch_url(url)

fetch_batch_urls(URLS)

CPU times: user 3.99 s, sys: 531 ms, total: 4.52 s
Wall time: 39 s


In [55]:
%%time

N = 100
URL = 'https://ru.wikipedia.org/wiki/Python'
URLS = [URL] * N
N_THREADS = 10
COUNT_SIZE = N // N_THREADS

import requests

def fetch_url(url):
    resp = requests.get(url)
    return resp
              
def fetch_batch_urls(urls):
    print(f"[{threading.current_thread().name}] Starting to fetch batch of size {len(urls)}...")
    for url in urls:
        fetch_url(url)
    print(f"[{threading.current_thread().name}] Finished...")


threads = [
    threading.Thread(target=fetch_batch_urls,
                     args=([URLS[i*COUNT_SIZE:(i+1)*COUNT_SIZE]]),
                     name=f"URL_FETCHER_THREAD#{i}")
    for i in range(N_THREADS)
]


for th in threads:
    th.start()

for th in threads:
    th.join()

[URL_FETCHER_THREAD#0] Starting to fetch batch of size 10...
[URL_FETCHER_THREAD#1] Starting to fetch batch of size 10...
[URL_FETCHER_THREAD#2] Starting to fetch batch of size 10...
[URL_FETCHER_THREAD#3] Starting to fetch batch of size 10...
[URL_FETCHER_THREAD#4] Starting to fetch batch of size 10...
[URL_FETCHER_THREAD#5] Starting to fetch batch of size 10...
[URL_FETCHER_THREAD#6] Starting to fetch batch of size 10...
[URL_FETCHER_THREAD#7] Starting to fetch batch of size 10...
[URL_FETCHER_THREAD#8] Starting to fetch batch of size 10...
[URL_FETCHER_THREAD#9] Starting to fetch batch of size 10...
[URL_FETCHER_THREAD#0] Finished...
[URL_FETCHER_THREAD#1] Finished...
[URL_FETCHER_THREAD#2] Finished...
[URL_FETCHER_THREAD#5] Finished...
[URL_FETCHER_THREAD#9] Finished...
[URL_FETCHER_THREAD#7] Finished...
[URL_FETCHER_THREAD#4] Finished...
[URL_FETCHER_THREAD#3] Finished...
[URL_FETCHER_THREAD#6] Finished...
[URL_FETCHER_THREAD#8] Finished...
CPU times: user 5.81 s, sys: 1.12 s, tot

In [68]:
N = 1_000
N_THREADS = 100
COUNT_SIZE = N // N_THREADS

count = [0]

lock = threading.Lock()

def count_operations(l, r, lock):
    for i in range(l, r):
        lock.acquire()
        tmp = count[0]
        tmp += 1
        y = tmp ** 10000
        for _ in range(tmp):
            a = y
        count[0] = tmp
        lock.release()

threads = [
    threading.Thread(target=count_operations,
                     args=(i*COUNT_SIZE, (i+1)*COUNT_SIZE, lock),
                     name=f"COUNT_OPERATIONS_THREAD#{i}")
    for i in range(N_THREADS)
]


for th in threads:
    th.start()

for th in threads:
    th.join()

print(count[0])

1000
